In [2]:
import gensim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.cross_validation import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support

In [3]:
documents=gensim.models.doc2vec.TaggedLineDocument('Data/itcomments.txt')

In [15]:
model = gensim.models.Doc2Vec(documents, size=200, window=8, workers=4, min_count=3, iter=10)
#size=100,window=8,min_count=3 SVC(kernal=linear) Acc ~ 7%
#size=200,window=8,min_count=3 SVC(kernal=linear) Acc ~ 7.5%
#size=200,window=8,min_count=3 SVC(kernal=linear) Acc ~ 8%

In [16]:
model.most_similar('BLUETOOTH')

[(u'BLUTOOTH', 0.726509153842926),
 (u'INTERNET', 0.6158711910247803),
 (u'USB', 0.45720401406288147),
 (u'HANDSFREE', 0.44516488909721375),
 (u'WIFI', 0.44510382413864136),
 (u'SERVICE', 0.4282341003417969),
 (u'CELLULAR', 0.42741823196411133),
 (u'ONSTAR', 0.4227968156337738),
 (u'IPOD', 0.4140186607837677),
 (u'CHARGING', 0.4032343029975891)]

In [17]:
vecList=[]
for i in xrange(len(model.docvecs)):
    vecList.append(model.docvecs[i])
    
a=np.asarray(vecList)
print(len(a))

24192


In [18]:
dataset=pd.read_csv("IT comments.csv")
col=dataset.columns.values
catList=[]
data=dataset.values
for i in range(24192):
    l=[]
    for j in range(len(col)):
        if data[i,j]==1:
            l.append(col[j])
    catList.append(l)
print(catList[0:2])
m=MultiLabelBinarizer()
y=m.fit_transform(catList)
print(y[:2])

[['Media Inputs', 'Amenities', 'IT'], ['Navigation', 'IT']]
[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0]]


In [47]:
y=dataset["Air Bags"].values
Xtrain, Xtest, ytrain, ytest=train_test_split(a,y,test_size=.66)
classifier = OneVsRestClassifier(LogisticRegression())
classifier.fit(Xtrain,ytrain)
precision_recall_fscore_support(y, classifier.predict(a))

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([ 0.99710648,  0.        ]),
 array([ 1.,  0.]),
 array([ 0.99855114,  0.        ]),
 array([24122,    70]))

In [44]:
print(classifier.score(Xtest,ytest))
#Random Forest ~ 5%
#Logistic Regression ~ 17%
#LDA ~ 18%
#LinearSVC ~ 16%
#GaussianNB ~ 9%
#SVC ~ 7%

0.76413853573
